In [11]:
import pandas as pd
import re
data = pd.read_csv('data/date_parser_testcases.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Input            99 non-null     object
 1   Expected Output  99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


,Input,Expected Output
0,"The event will take place on March 5, 2023.",05-03-2023
1,Her birthday is on 07/08/1990.,07-08-1990
2,The deadline is 2022-12-31.,31-12-2022
3,We met on 1st of January 2000.,01-01-2000
4,"The concert is scheduled for 15th September, 2...",15-09-2021


In [41]:
def create_date(out, key):
    month_id = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12', 'Dec': '12'}
    if key == 'standard':
        return re.sub(r'\D', '/', out)
    elif key == 'reversed':
        return "/".join(reversed(re.findall(r'\d+', out)))
    elif key == 'short':
        dmy =  re.findall(r'\d+', out)
        day = f"0{dmy[0]}" if len(dmy[0]) == 1 else dmy[0]
        month = f"0{dmy[1]}" if len(dmy[1]) == 1 else dmy[1]
        year = f"20{dmy[2]}" if int(dmy[2])<30 else f"19{dmy[2]}" # 2030 is the cutoff year
        return f"{day}/{month}/{year}"
    elif key == 'month day, yyyy':
        month = month_id[out.split()[0]]
        day = re.search(r'\d{1,2}', out.split()[1]).group()
        day = f"0{day}" if len(day) == 1 else day
        year = out.split()[2]
        return f"{day}/{month}/{year}"
    elif key == 'day month yyyy':
        out = out.replace(',', '')
        day = re.search(r'\d{1,2}', out.split()[0]).group()
        day = f"0{day}" if len(day) == 1 else day
        month = month_id[out.split()[-2]] # -2 since a word like on or of could occupy index 1
        year = out.split()[-1]
        return f"{day}/{month}/{year}"
    elif key == 'other':
        day = re.search(r'\d{1,2}', out.split()[0]).group()
        day = f"0{day}" if len(day) == 1 else day
        month = month_id[out.split()[-2]]
        year = out.split()[-1]
        return f"{day}/{month}/{year}"



In [42]:
def parse_date(input):
    patterns = {'standard': r"\d{2}.\d{2}.\d{4}",
            'reversed': r"\d{4}.\d{2}.\d{2}",
            'short': r'\d{1,2}.\d{1,2}.\d{2}',
            'month day, yyyy': r'(January|February|March|April|May|June|July|August|September|October|November|December|Dec)+\s+\d{1,2}(?:st|nd|rd|th)?.\s\d{4}',
            'day month yyyy': r'\d{1,2}(?:st|nd|rd|th)\s?(?:\w+\s*)?(January|February|March|April|May|June|July|August|September|October|November|December|Dec).?\s+\d{4}'}
    
    for key, pattern in patterns.items():
        out = re.search(pattern, input)
        if out:
            return create_date(out.group(), key)
        
    dm = re.search(r'\d{1,2}(?:st|nd|rd|th)\s?(?:\w+\s*)?(January|February|March|April|May|June|July|August|September|October|November|December|Dec)', input)
    year = re.search(r'\d{4}', input)
    if dm and year:
        out = ' '.join([dm.group(), year.group()])
        return create_date(out, 'other')
            
        
    return None


In [43]:
def test(test_cases):
    fails = []
    date = None
    for i in range(len(test_cases)):
        txt = test_cases['Input'][i]
        date = parse_date(txt)
        if date is None and date != test_cases['Expected Output'][i]:
            fails.append(i)

    return fails, len(fails)     


In [44]:
fail, n = test(data)
print(100-n, 'test cases passed')
### LOG
## n_patterns = n_test cases passed
# 2 = 54
# 3 = 60
# 4 = 74
# 5 = 98
# 6 = 100

100 test cases passed


In [37]:
# Viewing failed test cases
data.iloc[fail]

,Input,Expected Output
58,"We celebrate Christmas every year on 25th Dec,...",25-12-2024
66,We celebrate Independence Day on the 4th of Ju...,04-07-2022
